In [ ]:
!mkdir -p files/retailrocket \
  && curl -L -o files/retailrocket/ecommerce-dataset.zip https://www.kaggle.com/api/v1/datasets/download/retailrocket/ecommerce-dataset \
  && unzip files/retailrocket/ecommerce-dataset.zip -d files/retailrocket \
  && rm files/retailrocket/ecommerce-dataset.zip

In [2]:
import polars as pl

In [4]:
df_items = (
    pl.concat(
        [
            pl.read_csv(
                "files/retailrocket/item_properties_part1.csv",
                has_header=True,
                columns=["timestamp", "itemid", "property", "value"]
            ).filter(pl.col("property") == "categoryid").select("timestamp", pl.col("itemid").alias("item_id"), pl.col("value").cast(pl.Int32).alias("category_id")),
            pl.read_csv(
                "files/retailrocket/item_properties_part2.csv",
                has_header=True,
                columns=["timestamp", "itemid", "property", "value"]
            ).filter(pl.col("property") == "categoryid").select("timestamp", pl.col("itemid").alias("item_id"), pl.col("value").cast(pl.Int32).alias("category_id"))
        ],
        rechunk=True
    )
    .sort("item_id", "timestamp")
)

In [5]:
df_events = (
    pl.read_csv(
        "files/retailrocket/events.csv", 
        has_header=True, 
        columns=["timestamp", "visitorid", "event", "itemid"], 
        new_columns=["timestamp", "user_id", "behavior", "item_id"]
    )
    .sort("item_id", "timestamp")
    # this will raise a warning, but is actually correct as both dataframes are sorted within each group. 
    # see: https://github.com/pola-rs/polars/issues/21051    
    .join_asof(df_items, by="item_id", on="timestamp", strategy="backward")  
)

/var/folders/f1/pxc4yf2j27b6wkchwv3p6nj00000gp/T/ipykernel_32255/4163469148.py:11: UserWarning: left key of asof join is not sorted.

This can lead to invalid results. Ensure the asof key is sorted
  .join_asof(df_items, by="item_id", on="timestamp", strategy="backward")
/var/folders/f1/pxc4yf2j27b6wkchwv3p6nj00000gp/T/ipykernel_32255/4163469148.py:11: UserWarning: right key of asof join is not sorted.

This can lead to invalid results. Ensure the asof key is sorted
  .join_asof(df_items, by="item_id", on="timestamp", strategy="backward")


In [6]:
ts_cutoff = (
    df_events
    .select("timestamp")
    .quantile(0.9)
)['timestamp'][0]

In [10]:
df_train = df_events.filter(pl.col("timestamp") < ts_cutoff)
df_test_users = (
    df_events
    .group_by("user_id")
    .agg(pl.max("timestamp").alias("latest_appearance"))
    .filter(pl.col("latest_appearance") >= ts_cutoff)
)

df_test = df_events.join(df_test_users, "user_id")

test_dataset_per_user_cutoff = (
    df_test
    .filter(pl.col("timestamp") >= ts_cutoff)
    .sort(["user_id", "timestamp"])
    .with_columns(
        pl.col("timestamp").cast(pl.Int64).rank(method="ordinal", descending=False).over("user_id").alias("rank"),
        pl.col("timestamp").alias("ts_cutoff")
    )
    .filter(pl.col("rank") == 1)
    .select(["user_id", "ts_cutoff"])
)

df_test = (
    df_test
    .join(test_dataset_per_user_cutoff, "user_id")
    .filter(pl.col("timestamp") <= pl.col("ts_cutoff"))
)

In [11]:
item_mapping = (
    df_train
    .group_by("item_id")
    .agg(pl.count("item_id").alias("num_occurrences"))
    .filter(pl.col("num_occurrences") > 10) # retaining roughly 10% of items
    .sort("num_occurrences", descending=True)
    .with_columns(pl.col("num_occurrences").rank(descending=True, method="ordinal").alias("rank_item"))
)


In [12]:
item_mapping["rank_item"].max()

46893

In [13]:
category_mapping = (
    df_train
    .group_by("category_id")
    .agg(pl.count("category_id").alias("num_occurrences"))
    .filter(pl.col("num_occurrences") > 10)
    .sort("num_occurrences", descending=True)
    .with_columns(pl.col("num_occurrences").rank(descending=True, method="ordinal").alias("rank_category"))
)

In [14]:
category_mapping["rank_category"].max()

1006

In [ ]:
(
    df_train
    .with_columns(behavior=pl.col("behavior").replace_strict(["view", "addtocart", "transaction"], [0, 1, 2]))
    .join(item_mapping, on="item_id")
    .join(category_mapping, on="category_id")
    .sort(["user_id", "timestamp"])
    .group_by("user_id")
    .agg(
        pl.col("rank_item").alias("items"),
        pl.col("rank_category").alias("categories"),
        pl.col("behavior").alias("behaviors"),
        pl.col("timestamp").alias("timestamps"),
    )
    .sample(fraction=1.0, shuffle=True)
    .write_parquet("files/train_retailrocket_preprocessed.parquet")
)

In [ ]:
(
    df_test
    .with_columns(behavior=pl.col("behavior").replace_strict(["view", "addtocart", "transaction"], [0, 1, 2]))
    .join(item_mapping, on="item_id")
    .join(category_mapping, on="category_id")
    .sort(["user_id", "timestamp"])
    .group_by("user_id")
    .agg(
        pl.col("rank_item").alias("items"),
        pl.col("rank_category").alias("categories"),
        pl.col("behavior").alias("behaviors"),
        pl.col("timestamp").alias("timestamps"),
    )
    .write_parquet("files/test_retailrocket_preprocessed.parquet")
)

In [2]:
import polars as pl
train_dataset = pl.read_parquet("files/train_retailrocket_preprocessed.parquet")

In [2]:
(
    train_dataset
    .with_columns(
        pl.col("items").list.len().alias("num_items"),
        #add column, showing if len(items) == 1
        pl.when(pl.col("items").list.len() == 1)
        .then(pl.lit(1))
        .otherwise(pl.lit(0))
        .alias("is_single_item")
    )
    .select(
        pl.median("num_items"),
        pl.mean("is_single_item")
    )
)

num_items,is_single_item
f64,f64
1.0,0.701646
